In [2]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

#GeoJSON stadsdelen Amsterdam
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

#Get Sensors from data
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"],  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"], errors="coerce")

#Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

#Crowd sensor data used 
cs = pd.read_csv("sensordata_SAIL2025.csv")
cs_long = cs.melt(
    id_vars=["timestamp", "hour", "minute", "day", "month", "weekday", "is_weekend"],
    var_name="sensor",
    value_name="value"
)

cs_long = cs_long.sort_values("sensor").reset_index(drop=True)


print(cs_long.head())


#HEATMAP
fig = px.density_mapbox(
    df,
    lat="Lat",
    lon="Long",
    z="Persons",                     # gewicht: meer persons = hogere intensiteit
    radius=60,                       # straal in meters (speel hiermee!)
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd", # geel→rood
    opacity=0.7,
    height=700
)

fig.update_layout(
    mapbox_style="carto-positron",   # of "open-street-map" als je geen token hebt
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()


                   timestamp  hour  minute  day  month  weekday  is_weekend  \
0  2025-08-20 00:00:00+02:00     0       0   20      8        2           0   
1  2025-08-23 07:48:00+02:00     7      48   23      8        5           1   
2  2025-08-23 07:51:00+02:00     7      51   23      8        5           1   
3  2025-08-23 07:54:00+02:00     7      54   23      8        5           1   
4  2025-08-23 07:57:00+02:00     7      57   23      8        5           1   

           sensor  value  
0  CMSA-GAKH-01_0     15  
1  CMSA-GAKH-01_0      0  
2  CMSA-GAKH-01_0      1  
3  CMSA-GAKH-01_0      1  
4  CMSA-GAKH-01_0      0  


C:\Users\matsj\AppData\Local\Temp\ipykernel_27628\3839820215.py:38: DeprecationWarning: *density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.density_mapbox(


ValueError: Value of 'z' is not the name of a column in 'data_frame'. Expected one of ['Objectummer', 'Locatienaam', 'Lat/Long', 'Breedte', 'Effectieve breedte', 'Lat', 'Long'] but received: Persons

In [ ]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

#GeoJSON stadsdelen Amsterdam
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

#Get Sensors from data
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"],  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"], errors="coerce")

#Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

#Crowd sensor data used 
first_timestamp = cs["timestamp"].iloc[0]
cs_first = cs[cs["timestamp"] == first_timestamp]
time_cols = ["timestamp", "hour", "minute", "day", "month", "weekday", "is_weekend"]
sensor_cols = [c for c in cs.columns if c not in time_cols]
cs_first_long = cs_first[sensor_cols].melt(var_name="sensor", value_name="value").sort_values("sensor")
print(cs_first_long.head())

df["Persons"] = cs_first_long["value"]
#

#HEATMAP
fig = px.density_mapbox(
    df,
    lat="Lat",
    lon="Long",
    z="Persons",                     # gewicht: meer persons = hogere intensiteit
    radius=60,                       # straal in meters (speel hiermee!)
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd", # geel→rood
    opacity=0.7,
    height=700
)

fig.update_layout(
    mapbox_style="carto-positron",   # of "open-street-map" als je geen token hebt
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()


             sensor  value
0    CMSA-GAKH-01_0     15
1  CMSA-GAKH-01_180      4
2  CMSA-GAWW-11_120     29
3  CMSA-GAWW-11_300     33
4  CMSA-GAWW-12_115     44


C:\Users\matsj\AppData\Local\Temp\ipykernel_27628\1043097888.py:35: DeprecationWarning: *density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.density_mapbox(
